# Machine Learning - Deep Learning

In this homework, we'll build a model for predicting if we have an image of a bee or a wasp. For this, we will use the "Bee or Wasp?" dataset that was obtained from Kaggle and slightly rebuilt.


In [ ]:
# get the dataset

# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
# unzip data.zip

## Data Preparation

The dataset contains around 2500 images of bees and around 2100 images of wasps.

The dataset contains separate folders for training and test sets.

## Model
For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

- The shape for input should be (150, 150, 3)
- Next, create a convolutional layer (Conv2D):
  - Use 32 filters
  - Kernel size should be (3, 3) (that's the size of the filter)
  - Use 'relu' as activation
- Reduce the size of the feature map with max pooling (MaxPooling2D)
 - Set the pooling size to (2, 2)
- Turn the multi-dimensional result into vectors using a Flatten layer
- Next, add a Dense layer with 64 neurons and 'relu' activation
- Finally, create the Dense layer with 1 neuron - this will be the output
 - The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use SGD with the following parameters:
- SGD(lr=0.002, momentum=0.8)

In [14]:
# build the model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import History
import numpy as np


def build_model():

    model = Sequential([
        # Convolutional layer
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        # MaxPooling layer
        MaxPooling2D(pool_size=(2, 2)),
        # Flatten layer
        Flatten(),        
        # Dense layer with 64 neurons and 'relu' activation
        Dense(64, activation='relu'),
        # Output layer with 1 neuron and 'sigmoid' activation for binary classification
        Dense(1, activation='sigmoid')
    ])
  
    # Define the SGD optimizer with the specified parameters
    optimizer = SGD(learning_rate=0.002, momentum=0.8)

    # Compile the model with binary crossentropy loss and the defined optimizer
    model.compile(optimizer=optimizer, 
                 loss='binary_crossentropy', 
                 metrics=['accuracy'])

    # Print the model summary
    model.summary()

    return model

## Question 1 - Since we have a binary classification problem, what is the best loss function for us?

- mean squared error
- binary crossentropy
- categorical crossentropy
- cosine similarity

For binary classification problems, the most commonly used loss function is binary crossentropy. Binary crossentropy is suitable when you have a binary outcome (e.g., 0 or 1) and is particularly well-suited for models outputting probabilities.

##  Question 2 - What's the number of parameters in the convolutional layer of our model? You can use the summary method for that.

- 1
- 65
- 896
- 11214912

In [9]:
model = build_model()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 175232)            0         
                                                                 
 dense_4 (Dense)             (None, 64)                11214912  
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
____________

## Generators and Training
For the next two questions, use the following data generator for both train and test sets:


```python
ImageDataGenerator(rescale=1./255)

```

-We don't need to do any additional pre-processing for the images.
-When reading the data from train/test directories, check the class_mode parameter. Which value should it be for a binary classification problem?
-Use batch_size=20
-Use shuffle=True for both training and test sets.

For training use .fit() with the following params:

```python

model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

## Question 3 - What is the median of training accuracy for all the epochs for this model?

- 0.20
- 0.40
- 0.60
- 0.80

In [12]:
# Create an ImageDataGenerator for both training and test sets
datagen = ImageDataGenerator(rescale=1./255)

def get_data_gen(datagen, path):
    """
    Create an ImageDataGenerator and return the data generators for the training and test sets.
    """
    
    # Set up the data generators for the training set
    generator = datagen.flow_from_directory(
        path,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',  # Use 'binary' for binary classification
        shuffle=True
    )

    return generator

# define the path to the training and test directories
train_dir = './train'
test_dir = './test'

# Set up the data generators for the training set
train_generator = get_data_gen(datagen, train_dir)
    
# Set up the data generators for test set
test_generator = get_data_gen(datagen, test_dir)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

Found 3677 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
Epoch 1/10
184/184 [==============================] - 18s 95ms/step - loss: 0.6730 - accuracy: 0.5592
Epoch 2/10
184/184 [==============================] - 17s 94ms/step - loss: 0.6346 - accuracy: 0.6220
Epoch 3/10
184/184 [==============================] - 18s 95ms/step - loss: 0.6022 - accuracy: 0.6726
Epoch 4/10
184/184 [==============================] - 18s 97ms/step - loss: 0.5500 - accuracy: 0.7158
Epoch 5/10
184/184 [==============================] - 17s 92ms/step - loss: 0.5172 - accuracy: 0.7558
Epoch 6/10
184/184 [==============================] - 17s 91ms/step - loss: 0.4817 - accuracy: 0.7811
Epoch 7/10
184/184 [==============================] - 18s 98ms/step - loss: 0.4526 - accuracy: 0.8009
Epoch 8/10
184/184 [==============================] - 18s 95ms/step - loss: 0.4243 - accuracy: 0.8240
Epoch 9/10
184/184 [==============================] - 17s 92ms/step - loss: 0.3977 - accuracy: 0.8284
E

In [17]:
# Get the training accuracy values for each epoch
training_accuracy_values = history.history['accuracy']

# Calculate the median of training accuracy
median_training_accuracy = round(np.median(training_accuracy_values),2)

# Print the median training accuracy
print("Median Training Accuracy:", median_training_accuracy)

Median Training Accuracy: 0.77


## Question 4 - What is the standard deviation of training loss for all the epochs for this model?

- 0.031
- 0.061
- 0.091
- 0.131

In [18]:
# Get the training loss values for each epoch
training_loss_values = history.history['loss']

# Calculate the standard deviation of training loss
std_dev_training_loss = round(np.std(training_loss_values),3)

# Print the standard deviation of training loss
print("Standard Deviation of Training Loss:", std_dev_training_loss)

Standard Deviation of Training Loss: 0.099


## Data Augmentation

For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:

- rotation_range=50,
- width_shift_range=0.1,
- height_shift_range=0.1,
- zoom_range=0.1,
- horizontal_flip=True,
- fill_mode='nearest'

In [19]:

# Create an augmented ImageDataGenerator 
augmented_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

augmented_train_generator = get_data_gen(augmented_train_datagen, train_dir)

Found 3677 images belonging to 2 classes.


In [20]:
# Train the model with the augmented data generator
aug_history = model.fit(
    augmented_train_generator,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
184/184 [==============================] - 31s 171ms/step - loss: 0.5308 - accuracy: 0.7460
Epoch 2/10
184/184 [==============================] - 29s 158ms/step - loss: 0.5033 - accuracy: 0.7664
Epoch 3/10
184/184 [==============================] - 29s 159ms/step - loss: 0.4951 - accuracy: 0.7667
Epoch 4/10
184/184 [==============================] - 29s 159ms/step - loss: 0.4928 - accuracy: 0.7762
Epoch 5/10
184/184 [==============================] - 30s 162ms/step - loss: 0.4789 - accuracy: 0.7767
Epoch 6/10
184/184 [==============================] - 30s 162ms/step - loss: 0.4840 - accuracy: 0.7718
Epoch 7/10
184/184 [==============================] - 31s 169ms/step - loss: 0.4757 - accuracy: 0.7778
Epoch 8/10
184/184 [==============================] - 33s 181ms/step - loss: 0.4732 - accuracy: 0.7797
Epoch 9/10
184/184 [==============================] - 31s 167ms/step - loss: 0.4676 - accuracy: 0.7873
Epoch 10/10
184/184 [==============================] - 33s 180ms/step - l

## Question 5 - Let's train our model for 10 more epochs using the same code as previously.

> Note: make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.18
- 0.48
- 0.78
- 0.108

In [22]:
# get the mean of all the epochs for the test loss for the augmented model
aug_test_loss = round(np.mean(aug_history.history['loss']), 2)

# print the mean of the test loss for the aumented model
print("Mean of the Test Loss for the Augmented Model:", aug_test_loss)

Mean of the Test Loss for the Augmented Model: 0.49


## Question 6 - What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

- 0.38
- 0.58
- 0.78
- 0.98

In [23]:
# average of epochs 6 to 10 for the test accuracy for the augmented model 
aug_test_accuracy = round(np.mean(aug_history.history['accuracy'][5:]), 2)

# print the average of epochs 6 to 10 for the test accuracy for the augmented model
print("Average of the Test Accuracy for the Augmented Model:", aug_test_accuracy)


Average of the Test Accuracy for the Augmented Model: 0.78
